In [1]:
import nest_asyncio
nest_asyncio.apply()

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import hddCRP.dataLoader as dl
import hddCRP.newContextModel2 as crp
import os
import scipy.stats as stats 

In [2]:
print(dl.grp_names_all)

results_directory = "Results/populationNew2/"
if(not os.path.exists(results_directory)):
    os.makedirs(results_directory)

OVERWRITE = True
N_trials = 50

['diverse_TH', 'diverse_HT', 'uniform_H', 'uniform_T', 'diverse', 'uniform']


In [3]:
group = 'diverse'
model = crp.create_pop_model(group, n_trials=N_trials)

timescale = np.inf
context_a = 0.75;
context_b = 0.75
alpha = np.ones((3))*5/3
subject_similarity = 0

lls = model.compute_log_likelihood(timescale=timescale, context_a=context_a, context_b=context_b, alpha=alpha, subject_similarity=subject_similarity)

lls.sum(axis=0)

array([-54.44178094, -55.81388969, -55.94093822, -54.17326767,
       -53.99380047, -55.4404435 , -53.85050134, -56.36123022,
       -49.05872917])

In [4]:
# P = {}
# for group in ['uniform', 'diverse']:
#     model = crp.create_pop_model(group, n_trials=N_trials)
#     timescale = np.inf
#     context_b = 1.0;
#     alpha = np.ones((3))*6/3
#     subject_similarity = 0.0

#     context_as = np.linspace(0,1,20)
#     lls = np.zeros((context_as.size, model.K))


#     for ii, context_a in enumerate(context_as):

#         ll_c = model.compute_log_likelihood(timescale=timescale, context_a=context_a, context_b=context_b, alpha=alpha, subject_similarity=subject_similarity)

#         lls[ii,:] = ll_c.sum(axis=0)

#     P[group] = lls


In [5]:
# d_div =P["diverse"][-1,:] - P["diverse"][0,:]
# d_uni = P["uniform"][-1,:] - P["uniform"][0,:]
# # plt.plot(context_as,lls)
# plt.figure(figsize=(12,8))
# plt.subplot(2,3,1)
# plt.plot(context_as, P["uniform"].mean(axis=1),'b')
# plt.plot(context_as, P["diverse"].mean(axis=1),'r')
# plt.plot(context_as, np.ones_like(context_as)*np.log(1/3)*N_trials,'k:')
# plt.subplot(2,3,2)
# plt.plot(context_as, P["uniform"])
# plt.plot(context_as, np.ones_like(context_as)*np.log(1/3)*N_trials,'k:')
# plt.subplot(2,3,3)
# plt.plot(context_as, P["diverse"])
# plt.plot(context_as, np.ones_like(context_as)*np.log(1/3)*N_trials,'k:')
# plt.subplot(2,3,5)
# plt.hist(d_uni)
# plt.subplot(2,3,6)
# plt.hist(d_div)

# stats.mannwhitneyu(d_div, d_uni)

In [16]:
L = {}

a1 = 1
a2 = 1

for group in ['uniform_T', 'uniform_H', 'diverse_HT', 'diverse_TH']:
    model = crp.create_pop_model(group, n_trials=N_trials)
    ll_c = np.zeros((model.K, 4))

    context_a = 1
    context_b = 1
    ll_c[:,0] = model.compute_cross_val_test(context_a, context_b, a1, a2)
    context_a = 1
    context_b = 0
    ll_c[:,1] = model.compute_cross_val_test(context_a, context_b, a1, a2)
    context_a = 0
    context_b = 1
    ll_c[:,2] = model.compute_cross_val_test(context_a, context_b, a1, a2)
    context_a = 0
    context_b = 0
    ll_c[:,3] = model.compute_cross_val_test(context_a, context_b, a1, a2)

array([-53.88783387, -52.8876574 , -54.85767196, -53.72253514,
       -60.56867119])

In [7]:
raise RuntimeError
fit_file = f"{results_directory}/fit"
fit_summary_file = f"{results_directory}/fit_summary"
fit_file += f".pkl"
fit_summary_file  += f".pkl"

if(((not os.path.isfile(fit_file)) or (not os.path.isfile(fit_summary_file))) or OVERWRITE):
    data_fits = pd.DataFrame()
    data_fit_metrics = pd.DataFrame()
else:
    data_fits = pd.read_pickle(fit_file)
    data_fit_metrics = pd.read_pickle(fit_summary_file)


for ii, group in enumerate(dl.grp_names_all):
    if( ("group" in data_fit_metrics) and data_fit_metrics.query("group == @group").size > 0):
        print(f"found group {ii}: {group}")
        continue
    
    print(f"fitting group {ii}: {group}")
    rs = 1200 + ii;
    model = crp.create_pop_model(group, n_trials=N_trials)
    model.build(random_seed = rs)
    model.fit_model()

RuntimeError: 